*Stanislav Borysov [stabo@dtu.dk], DTU Management*
# Advanced Business Analytics

## Text Analytics - Part 1: Spam Classification

Given that language remains a primary means of communication between humans, a lot of extremely useful data come in a natural language form. However, given its complexity and ambiguity, representing these data in a suitable form for automatic processing is a very difficult research challenge. In this exercise, we will get familiar with some basic text analysis techniques. As an exercise, we will try to classify SMS messages for spam detection. The data was downloaded from http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/ and the notebook is partially based on [this](https://www.kaggle.com/dejavu23/sms-spam-or-ham-beginner) Kaggle kernel.

### 1. Descriptive data analysis

As usual in data science, we will start with descriptive data analysis to understand the data structure and get useful insights. Load the data from `smsspamcollection/SMSSpamCollection.txt` and print the number of spam and non-spam ("ham") messages.

In [ ]:
# ...

Note that the data is imbalanced, which is very common in such types of problems. Now, plot histogram of message lengths and print average message length for each message type.

In [ ]:
# ...

The very first insight: The spam messages tend to be longer! It might be a useful feature in our classification problem. Let's visualize the most popular words for each category using word clouds. You can use either the `wordcloud` Python package or some external websites such as https://www.wordclouds.com/

In [ ]:
# ...

These word clouds suggest that using words as features for our classification problem can be a not bad idea.

### 2. Text preprocessing

First, we need to do some text preprocessing, commonly referred to as "normalization". Text preprocessing is usually the most crucial and time-consuming part of the NLP pipeline. In this exercise, it will include the following simple steps:

1. Removing punctuation
2. Converting text to lowercase *(Side note: This step is questionable since spammers tend to use more uppercase letters)*
3. Removing stopwords (either using a predefined list of the English stopwords or using the most frequent words in the text directly)
4. (Optional) Stemming or lemmatizing

This list is, of course, is incomplete and other steps can also include removing rare words, removing numbers, synonymizing, handling negation, etc.

In Python, there are many NLP packages available, such as `NLTK` (the most popular one), `Gensim`, or `spaCy`, however, we will mostly use the basic functionality provided by `sklearn`. For stemming or lemmatizing, one can use the `NLTK` library.

*Side note: In principle, all preprocessing should be done on the train and test sets separately. However, we will skip this for simplicity.*

In [ ]:
from sklearn.feature_extraction import stop_words
import string
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

#print(stop_words.ENGLISH_STOP_WORDS)
#print(string.punctuation)

In [ ]:
# ...

Print a few messages to check how the preprocessed messages differ from the raw ones.

In [ ]:
# ...

Let's now represent the text in a format which can be used in machine learning. Since we will use a vector format for the messages ("documents"), this process is called "vectorization". We will use a simple "bag-of-words" (BoW) representation on a single word level (1-grams). First, let's do simple word counts, where `sklearn.feature_extraction.text.CountVectorizer` can be particularly useful. Such vectorizers convert the whole corpora to a document-term matrix of size $N\times M$, where $N$ is a number of documents (messages), $M$ is a vocabulary size (number of different words in the corpora), and each element $d_{ij}$ corresponds to the number (or frequency, or tf-idf, etc) of the word $j$ in the document $i$. As each document is usually far from containing all the words from vocabulary, the matrix is usually very sparse (has a lot of 0's).

In [ ]:
# ...

Print the vocabulary size. It should be close to 9000 (this is a huge number of features!).

In [ ]:
# ...

Now let's weight the term frequencies by inverse document frequency (tf-idf representation). 

*Hint: `sklearn` has this functionality as well, check `sklearn.feature_extraction.text.TfidfTransformer`*

In [ ]:
# ...

Having our text normalized and vectorized, let's finally proceed to machine learning!

### 3. Classification

To classify messages as spam/ham (binary classification), we will compare three different sets of features: 
1. Length of messages only
2. Term frequencies (counts)
3. Tf-idf

We will also compare these classifiers to a naive baseline which predicts the most frequent class. Before starting, make sure that you have all the features and the target labels ready for the modelling.

In [ ]:
# ...

We will use 80%/20% train/test split. Note that as the data is imbalanced, this split should be stratified, i.e., the proportion of spam messages in the test set should be the same as in the whole dataset.

In [ ]:
# ...

Check if the proportion of the spam messages is the same in the test set and the whole dataset.

In [ ]:
# ...

**Naive baseline**

Always start with the simplest baseline, for example, predicting the most frequent class in the training set. Use standard performance metrics such as confusion matrix and F1 score.

In [ ]:
# ...

**Classification using the message length only**

Train classifier which uses the message length as the only feature. Use your favorite model (e.g. Logistic Regression, Support Vector Machine or Neural Net). Remember which implications the data imbalance might have for the model training (e.g., set `class_weight='balanced'` for `sklearn.linear_model.LogisticRegression`). Use standard performance metrics such as confusion matrix and F1 score.

*Optional: If you want, you can balance the training set directly, for instance, using upsampling or downsampling.*

*Optional: You can also tune hyperparameters (e.g. `C` in `sklearn.linear_model.LogisticRegression`), for example, using grid search on the training set based on cross-validation.*

In [ ]:
# ...

**Classification using term frequencies (counts)**

Train classifier which uses the term frequencies as features. Use standard performance metrics such as confusion matrix and F1 score.

In [ ]:
# ...

**Classification using tf-idf matrix**

Train classifier which uses the tf-idf matrix as features. Use standard performance metrics such as confusion matrix and F1 score.

In [ ]:
# ...

Which model performs better? Discuss your results with another student.

### 4. Topic modeling

Topic modeling is a useful set of techniques to represent a document as a set of concepts (or "topics"). Conceptually, it is similar to eigenvector decomposition (Principal Component Analysis, PCA), when each data point (document) is represented as a weighted sum of different eigenvectors (topics). Topics and decompositions are learned by the algorithm in an unsupervised manner. We will use the most popular one -- Latent Dirichlet Allocation (LDA). There are many implementations of the LDA algorithm in different languages and libraries, including `sklearn`.

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 7
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

Let's run the LDA algorithm. Note that it is designed to take term frequency matrix (counts) as an input, however, it is possible to run on tf-idf matrix as well.

In [ ]:
# ...

Print top N words together with their weights in each topic

In [ ]:
# ...

Do they have any meaning? Discuss with another student. Try to experiment with different numbers of topics and see how it affects their interpretability.


**Classification using LDA**

Train classifier which uses topic weights for each document (message) as features for the case `n_topics = 7`. Use standard performance metrics such as confusion matrix and F1 score.

In [ ]:
# ...

Not bad! You can get very good performance using only a small number of features for each message instead of thousands of words!

Finally, plot the classification performance for different numbers of topics. Also, inspect topics for the best performing model.

*Note: The topics number is itself a hyperparameter, which should be tuned using the training set only. However, as LDA can be computationally demanding, we will skip this part and calculate the performance for the different number of topics directly on the test set.*

In [ ]:
# ...